<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import math
import unittest
import cmath # For complex math like cmath.exp, cmath.log

class FunctionsTests(unittest.TestCase):

    def setUp(self):
        # Common setup if needed, e.g., for evaluation date, though not strictly necessary for these math tests.
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        ql.Settings.instance().evaluationDate = ql.Date(15, ql.May, 2020) # A default date

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testFactorial(self):
        print("Testing factorial numbers...")

        expected = 1.0
        # Factorial.get(n) in C++ maps to ql.Factorial.get(n)
        calculated = ql.Factorial.get(0)
        self.assertEqual(calculated, expected, f"Factorial(0) = {calculated}")

        for i in range(1, 171): # C++ loop is < 171, so Python range(1, 171)
            expected *= float(i) # Ensure expected remains float
            calculated = ql.Factorial.get(i)

            # Relative error check
            if expected == 0: # Avoid division by zero if expected is 0 (though not for factorial > 0)
                relative_error = abs(calculated - expected)
            else:
                relative_error = abs(calculated - expected) / expected

            self.assertLessEqual(relative_error, 1.0e-9,
                                 msg=(f"Factorial({i})\n"
                                      f"    calculated: {calculated:.16e}\n"
                                      f"    expected:   {expected:.16e}\n"
                                      f"    rel. error: {relative_error:.2e}"))

    def testGammaFunction(self):
        print("Testing Gamma function...")
        gamma_func = ql.GammaFunction()

        expected_log_gamma = 0.0
        # gamma_func.logValue(x)
        calculated_log_gamma = gamma_func.logValue(1.0)
        self.assertAlmostEqual(calculated_log_gamma, expected_log_gamma, delta=1.0e-15,
                               msg=(f"GammaFunction().logValue(1)\n"
                                    f"    calculated: {calculated_log_gamma:.16e}\n"
                                    f"    expected:   {expected_log_gamma:.16e}"))

        current_log_sum = 0.0 # To store log(i!) which is log(Gamma(i+1))
        for i in range(2, 9000): # C++ loop is < 9000
            # C++ expected:  expected  += std::log(Real(i));
            # This means expected is log( (i)! ) if loop starts from 1.
            # If loop starts from 2, expected accumulates log(2*3*...*i)
            # So, expected becomes log(i!)
            current_log_sum += math.log(float(i))
            expected_log_gamma_i_plus_1 = current_log_sum

            calculated_log_gamma_i_plus_1 = gamma_func.logValue(float(i + 1))

            if expected_log_gamma_i_plus_1 == 0:
                 relative_error = abs(calculated_log_gamma_i_plus_1 - expected_log_gamma_i_plus_1)
            else:
                relative_error = abs(calculated_log_gamma_i_plus_1 - expected_log_gamma_i_plus_1) / abs(expected_log_gamma_i_plus_1)

            self.assertLessEqual(relative_error, 1.0e-9,
                                 msg=(f"GammaFunction().logValue({i+1})\n"
                                      f"    calculated: {calculated_log_gamma_i_plus_1:.16e}\n"
                                      f"    expected:   {expected_log_gamma_i_plus_1:.16e}\n"
                                      f"    rel. error: {relative_error:.2e}"))

    def testGammaValues(self):
        print("Testing Gamma values...")
        gamma_func = ql.GammaFunction()

        # { x, expected_gamma_value, tolerance_multiplier_for_epsilon }
        tasks = [
            [0.0001, 9999.422883231624, 1e3],
            [1.2, 0.9181687423997607, 1e3],
            [7.3, 1271.4236336639089586, 1e3],
            [-1.1, 9.7148063829028946, 1e3],
            [-4.001, -41.6040228304425312, 1e3],
            [-4.999, -8.347576090315059, 1e3],
            [-19.000001, 8.220610833201313e-12, 1e8],
            [-19.5, 5.811045977502255e-18, 1e3],
            [-21.000001, 1.957288098276488e-14, 1e8],
            [-21.5, 1.318444918321553e-20, 1e6]
        ]

        for task_data in tasks:
            x, expected_val, tol_multiplier = task_data
            calculated_val = gamma_func.value(x)

            # C++ tolerance: tol_multiplier * QL_EPSILON * std::fabs(expected_val)
            # ql.QL_EPSILON is available in Python
            tolerance = tol_multiplier * ql.QL_EPSILON * abs(expected_val)

            self.assertAlmostEqual(calculated_val, expected_val, delta=tolerance,
                                   msg=(f"GammaFunction().value({x})\n"
                                        f"    calculated: {calculated_val:.16e}\n"
                                        f"    expected:   {expected_val:.16e}\n"
                                        f"    abs error:  {abs(calculated_val - expected_val):.2e}\n"
                                        f"    tolerance:  {tolerance:.2e}"))

    def testModifiedBesselFunctions(self):
        print("Testing modified Bessel function of first and second kind...")
        # { nu, x, expected_I, expected_K }
        real_bessel_tasks = [
            [-1.3, 2.0, 1.2079888436539505, 0.1608243636110430],
            [1.3, 2.0, 1.2908192151358788, 0.1608243636110430],
            [0.001, 2.0, 2.2794705965773794, 0.1138938963603362],
            [1.2, 0.5, 0.1768918783499572, 2.1086579232338192],
            [2.3, 0.1, 0.00037954958988425198, 572.096866928290183],
            [-2.3, 1.1, 1.07222017902746969, 1.88152553684107371],
            [-10.0001, 1.1, 13857.7715614282552, 69288858.9474423379]
        ]
        tol_multiplier = 5e4 # From C++ code for both I and K for real args

        for task_data in real_bessel_tasks:
            nu, x, expected_i, expected_k = task_data

            # ql.modifiedBesselFunction_i(nu, x)
            # ql.modifiedBesselFunction_k(nu, x)
            calculated_i = ql.modifiedBesselFunction_i(nu, x)
            calculated_k = ql.modifiedBesselFunction_k(nu, x)

            tol_i = tol_multiplier * ql.QL_EPSILON * abs(expected_i)
            tol_k = tol_multiplier * ql.QL_EPSILON * abs(expected_k)

            self.assertAlmostEqual(calculated_i, expected_i, delta=tol_i,
                                   msg=(f"Modified Bessel I(nu={nu}, x={x})\n"
                                        f"    calc: {calculated_i:.16e}, exp: {expected_i:.16e}"))
            self.assertAlmostEqual(calculated_k, expected_k, delta=tol_k,
                                   msg=(f"Modified Bessel K(nu={nu}, x={x})\n"
                                        f"    calc: {calculated_k:.16e}, exp: {expected_k:.16e}"))

        # Complex arguments
        # { nu, z_real, z_imag, exp_I_real, exp_I_imag, exp_K_real, exp_K_imag }
        complex_bessel_tasks = [
            [-1.3, 2.0, 0.0, 1.2079888436539505, 0.0, 0.1608243636110430, 0.0],
            [1.2, 1.5, 0.3, 0.7891550871263575, 0.2721408731632123, 0.275126507673411, -0.1316314405663727],
            [1.2, -1.5,0.0,-0.6650597524355781, -0.4831941938091643, -0.251112360556051, -2.400130904230102],
            [-11.2, 1.5, 0.3,12780719.20252659, 16401053.26770633, -34155172.65672453, -43830147.36759921],
            [1.2, -1.5,2.0,-0.3869803778520574, 0.9756701796853728, -3.111629716783005, 0.6307859871879062],
            [1.2, 0.0, 9.9999,-0.03507838078252647, 0.1079601550451466, -0.05979939995451453, 0.3929814473878203],
            [1.2, 0.0, 10.1, -0.02782046891519293, 0.08562259917678558, -0.02035685034691133, 0.3949834389686676],
            [1.2, 0.0, 12.1, 0.07092110620741207, -0.2182727210128104, 0.3368505862966958, -0.1299038064313366],
            [1.2, 0.0, 14.1,-0.03014378676768797, 0.09277303628303372, -0.237531022649052, -0.2351923034581644],
            [1.2, 0.0, 16.1,-0.03823210284792657, 0.1176663135266562, -0.1091239402448228, 0.2930535651966139],
            [1.2, 0.0, 18.1,0.05626742394733754, -0.173173324361983, 0.2941636588154642, -0.02023355577954348],
            [1.2, 0.0, 180.1,-0.001230682086826484, 0.003787649998122361, 0.02284509628723454, 0.09055419580980778],
            [1.2, 0.0, 21.0,-0.04746415965014021, 0.1460796627610969, -0.2693825171336859, -0.04830804448126782],
            [1.2, 10.0, 0.0, 2609.784936867044, 0.0, 1.904394919838336e-05, 0.0],
            [1.2, 14.0, 0.0, 122690.4873454286, 0.0, 2.902060692576643e-07, 0.0],
            [1.2, 20.0, 10.0, -37452017.91168936, -13917587.22151363, -3.821534367487143e-10, 4.083211255351664e-10],
            [1.2, 9.0, 9.0, -621.7335051293694,  618.1455736670332, -4.480795479964915e-05, -3.489034389148745e-08]
        ]
        tol_mult_i_complex = 5e4
        tol_mult_k_complex = 1e6

        for task_data in complex_bessel_tasks:
            nu_c, zr, zi, ei_r, ei_i, ek_r, ek_i = task_data
            z_complex = complex(zr, zi)
            expected_i_complex = complex(ei_r, ei_i)
            expected_k_complex = complex(ek_r, ek_i)

            calculated_i_complex = ql.modifiedBesselFunction_i(nu_c, z_complex)
            calculated_k_complex = ql.modifiedBesselFunction_k(nu_c, z_complex)

            # Use abs() for complex numbers to get magnitude for tolerance calc
            tol_i_c = tol_mult_i_complex * ql.QL_EPSILON * abs(expected_i_complex)
            tol_k_c = tol_mult_k_complex * ql.QL_EPSILON * abs(expected_k_complex)

            # Compare complex numbers by checking magnitude of difference
            self.assertLessEqual(abs(calculated_i_complex - expected_i_complex), tol_i_c,
                                 msg=(f"Modified Bessel I_complex(nu={nu_c}, z={z_complex})\n"
                                      f"    calc: {calculated_i_complex}, exp: {expected_i_complex}"))

            # C++ test: do not check small values for K
            if abs(expected_k_complex) > 1e-4:
                self.assertLessEqual(abs(calculated_k_complex - expected_k_complex), tol_k_c,
                                     msg=(f"Modified Bessel K_complex(nu={nu_c}, z={z_complex})\n"
                                          f"    calc: {calculated_k_complex}, exp: {expected_k_complex}"))

    def testWeightedModifiedBesselFunctions(self):
        print("Testing weighted modified Bessel functions...")
        # Test for real arguments
        nu_values_real = [n * 0.5 for n in range(-10, 11)] # -5.0 to 5.0 step 0.5
        x_values_real = [0.1 + i * 0.5 for i in range(30)] # 0.1 to 15.0 step 0.5 (approx)

        for nu in nu_values_real:
            for x in x_values_real:
                if x <= 0: continue # Bessel functions often undefined or complex for x <= 0

                calc_i_w = ql.modifiedBesselFunction_i_exponentiallyWeighted(nu, x)
                exp_i_w = ql.modifiedBesselFunction_i(nu, x) * math.exp(-x)

                calc_k_w = ql.modifiedBesselFunction_k_exponentiallyWeighted(nu, x)
                # Expected K_w based on relation K_nu(x) = pi/2 * (I_{-nu}(x) - I_{nu}(x)) / sin(pi*nu)
                # Ensure sin(pi*nu) is not zero (i.e., nu is not an integer) for this formula
                # QL might use different stable methods for integer nu.
                # The C++ test uses this formula directly.
                if abs(math.sin(math.pi * nu)) < 1e-9: # nu is close to an integer
                    # For integer nu, K_nu = K_{-nu}. The formula is indeterminate.
                    # QL's internal K calculation handles this. We test consistency.
                    exp_k_w = ql.modifiedBesselFunction_k(nu, x) * math.exp(-x)
                else:
                    exp_k_w = (math.pi / 2.0) * \
                              (ql.modifiedBesselFunction_i(-nu, x) - ql.modifiedBesselFunction_i(nu, x)) * \
                              math.exp(-x) / math.sin(math.pi * nu)

                tol_i_w = 1e3 * ql.QL_EPSILON * abs(exp_i_w) * max(math.exp(x), 1.0)
                tol_k_w = max(ql.QL_EPSILON, 1e3 * ql.QL_EPSILON * abs(exp_k_w) * max(math.exp(x), 1.0))

                self.assertAlmostEqual(calc_i_w, exp_i_w, delta=tol_i_w,
                                       msg=f"Weighted Bessel I_w(nu={nu}, x={x})")
                self.assertAlmostEqual(calc_k_w, exp_k_w, delta=tol_k_w,
                                       msg=f"Weighted Bessel K_w(nu={nu}, x={x})")

        # Test for complex arguments
        # C++ loops are fairly extensive. Let's use a smaller representative set for Python.
        nu_values_complex = [-2.0, 0.0, 1.5, 3.0]
        coord_values = [-2.0, 0.0, 1.0, 3.0] # For real and imag parts of z

        for nu_c in nu_values_complex:
            for x_c in coord_values:
                for y_c in coord_values:
                    z = complex(x_c, y_c)
                    if abs(z) < 1e-9: continue # Avoid z=0

                    calc_i_wc = ql.modifiedBesselFunction_i_exponentiallyWeighted(nu_c, z)
                    exp_i_wc = ql.modifiedBesselFunction_i(nu_c, z) * cmath.exp(-z) # Use cmath.exp

                    calc_k_wc = ql.modifiedBesselFunction_k_exponentiallyWeighted(nu_c, z)
                    if abs(math.sin(math.pi * nu_c)) < 1e-9: # nu is integer
                        exp_k_wc = ql.modifiedBesselFunction_k(nu_c, z) * cmath.exp(-z)
                    else:
                        exp_k_wc = (math.pi / 2.0) * \
                                   (ql.modifiedBesselFunction_i(-nu_c, z) * cmath.exp(-z) - \
                                    ql.modifiedBesselFunction_i(nu_c, z) * cmath.exp(-z)) / \
                                   math.sin(math.pi * nu_c)

                    # Tolerances as in C++ (relative to calculated value for weighted funcs)
                    tol_i_wc = 1e3 * ql.QL_EPSILON * abs(calc_i_wc)
                    tol_k_wc = 1e3 * ql.QL_EPSILON * abs(calc_k_wc)

                    # Compare complex numbers by magnitude of difference
                    self.assertLessEqual(abs(calc_i_wc - exp_i_wc), tol_i_wc,
                                         msg=f"Weighted Bessel I_wc(nu={nu_c}, z={z})")
                    self.assertLessEqual(abs(calc_k_wc - exp_k_wc), tol_k_wc,
                                         msg=f"Weighted Bessel K_wc(nu={nu_c}, z={z})")


    def testExpm1(self):
        print("Testing complex valued expm1...")
        # Test case 1: General complex number
        z1 = complex(1.2, 0.5)
        # ql.expm1(complex) should be available
        # QL_CHECK_SMALL(std::abs(std::exp(z) - 1.0 - expm1(z)), 10*QL_EPSILON);
        self.assertLessEqual(abs(cmath.exp(z1) - 1.0 - ql.expm1(z1)), 10 * ql.QL_EPSILON)

        # Test case 2: Small complex number (for precision of expm1)
        z2_calc = ql.expm1(complex(5e-6, 5e-5))
        z2_expected = complex(4.998762493771078e-06, 5.000024997979157e-05) # Scipy reference

        tol_expm1 = max(2.2e-14, 100 * ql.QL_EPSILON)
        # QL_CHECK_CLOSE_FRACTION compares (a-b)/max(a,b,threshold) <= tol
        # Python equivalent: assertAlmostEqual with a delta derived from relative tolerance
        # For simplicity, let's use assertAlmostEqual with a small absolute delta,
        # or calculate relative diff if values are not tiny.
        self.assertAlmostEqual(z2_calc.real, z2_expected.real, delta = tol_expm1 * abs(z2_expected.real),
                               msg="expm1 real part mismatch")
        self.assertAlmostEqual(z2_calc.imag, z2_expected.imag, delta = tol_expm1 * abs(z2_expected.imag),
                               msg="expm1 imag part mismatch")


    def testLog1p(self):
        print("Testing complex valued log1p...")
        # Test case 1: General complex number
        z1 = complex(1.2, 0.57)
        # ql.log1p(complex) should be available
        self.assertLessEqual(abs(cmath.log(1.0 + z1) - ql.log1p(z1)), 10 * ql.QL_EPSILON)

        # Test case 2: Small complex number
        z2_calc = ql.log1p(complex(5e-6, 5e-5))
        z2_expected = complex(5.0012374875401984e-06, 4.999974995958395e-05) # Scipy reference

        tol_log1p = max(2.2e-14, 100 * ql.QL_EPSILON)
        self.assertAlmostEqual(z2_calc.real, z2_expected.real, delta = tol_log1p * abs(z2_expected.real),
                               msg="log1p real part mismatch")
        self.assertAlmostEqual(z2_calc.imag, z2_expected.imag, delta = tol_log1p * abs(z2_expected.imag),
                               msg="log1p imag part mismatch")


if __name__ == '__main__':
    print("C++ test suite: FunctionsTests")
    print("Python QuantLib version: " + ql.__version__)
    print("-------------------------------------------------------")
    suite = unittest.TestSuite()
    suite.addTest(unittest.makeSuite(FunctionsTests))
    unittest.TextTestRunner(verbosity=2).run(suite)